In [2]:
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.display import display
from IPython.utils.capture import capture_output
from pprint import pprint
from pathlib import Path

In [5]:
@magics_class
class CaptureMagic(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "--path",
        "-p",
        default=None,
        help=(
            "The path where the video will be saved to. When there is more then one video, multiple paths have to be defined"
        ),
    )
    @cell_magic
    def capture_video(self, line, cell):
        args = magic_arguments.parse_argstring(CaptureMagic.capture_video, line)
        paths = args.path.strip('"').split(" ")
        with capture_output(stdout=False, stderr=False, display=True) as result:
            self.shell.run_cell(cell)
        for output in result.outputs:
            data = output.data

            #pprint(data) # for debugging 

            print("#####")
            if "text/html" in data:
                path = paths.pop(0)
                if not path:
                    raise ValueError("Too few paths given!")
                video_object = data["text/html"]
                split_string = video_object.split('"')
                video_url = split_string[1]
                print(video_url)
                print(path)

                dest = Path(path)
                src = Path(video_url)
                dest.write_bytes(src.read_bytes())
                
            display(output) # only disabled for debugging

                
ipy = get_ipython()
ipy.register_magics(CaptureMagic)

In [6]:
%%capture_video -p "Hello.mp4"
from IPython.display import Video
Video('DopplerTest.mp4')

#####
DopplerTest.mp4
Hello.mp4
